<a href="https://colab.research.google.com/github/SaiMudigonda/Saikumar_INFO5731_Spring2021/blob/main/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [10]:
# Write your code here
#PRE PROCESSING
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium



Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [11]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [12]:
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
import pandas as pd
import time

DRV = webdriver.Chrome('chromedriver',options=options)
link = 'https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
title_arr = []
review_arr = []
DRV.get(link)
for num in range(4):
  DRV.find_element_by_class_name("ipl-load-more__button").click()
  time.sleep(5)
  Titlelist = DRV.find_elements(By.CLASS_NAME, "title")
  Reviewslist = DRV.find_elements(By.CLASS_NAME, "text")
for ele, sub_ele in zip(Titlelist, Reviewslist):
  #appending title and review arrays
      title_arr.append((ele.text).replace('\n',''))
      review_arr.append(sub_ele.text)
df = pd.DataFrame(list(zip(title_arr, review_arr)), columns =['Title', 'Review'])
print("Length of data frame is {0}".format(len(df)))
df

Length of data frame is 125


,Title,Review
0,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul..."
1,Only certain people can relate,This is a movie that only those who have felt ...
2,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o..."
3,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...
4,The Hype is real,Most of the time movies are anticipated like t...
...,...,...
120,"Went into this blind, and i didnt enjoy it at ...",I didnt really know much about this movie. I h...
121,Great,
122,Very Disappointing,Film was directed and produced poorly. Opportu...
123,"By far the best DC movie to date, Joaquin Phoe...","Wow, what a movie! I have to admit, When I fir..."


In [13]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from nltk.stem import PorterStemmer
from textblob import Word
from textblob import TextBlob

nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
df['After removing noise'] = df['Review'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

     # Punctuation removal
df['Removal of Punctuation'] = df['After removing noise'].str.replace('[^\w\s]','')

     # Remove numbers
df['Remove numbers'] = df['Removal of Punctuation'].str.replace('\d+', '')

     # Stopwords removal
stop_word = stopwords.words('english')
df['Removal of Stopwords'] = df['Remove numbers'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_word))

     # Lower Casing
df['Lower casing'] = df['Removal of Stopwords'].apply(lambda x: " ".join(x.lower() for x in x.split()))

     # Stemming
st = PorterStemmer()
df['Stemming'] = df['Lower casing'].apply(lambda x: " ".join([st.stem(word) for word in x]))

    # Lemmatization
 
df['Lemmatization'] = df['Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

final_df = df

In [19]:
#1.1
nltk.download('punkt')
import itertools
list_of_words = []
for i in df['Lower casing']:
  list_of_words.append(nltk.tokenize.word_tokenize(i))
Words = [i for i in list_of_words if i != []]
looping = list(itertools.chain.from_iterable(Words))
#N-grams (N=3 -> Trigrams)
fd = nltk.FreqDist(nltk.trigrams(looping))
fd.most_common()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[(('the', 'dark', 'knight'), 7),
 (('movies', 'i', 'seen'), 4),
 (('super', 'hero', 'movie'), 3),
 (('i', 'ever', 'seen'), 3),
 (('clown', 'prince', 'crime'), 3),
 (('first', 'time', 'i'), 3),
 (('jack', 'nicholson', 'heath'), 3),
 (('nicholson', 'heath', 'ledger'), 3),
 (('performance', 'joaquin', 'phoenix'), 3),
 (('one', 'best', 'movies'), 3),
 (('what', 'i', 'say'), 3),
 (('robert', 'de', 'niro'), 3),
 (('mentally', 'ill', 'person'), 3),
 (('a', 'lot', 'people'), 2),
 (('one', 'best', 'films'), 2),
 (('best', 'films', 'decade'), 2),
 (('joaquin', 'phoenix', 'gives'), 2),
 (('oscar', 'winning', 'performance'), 2),
 (('joker', 'first', 'time'), 2),
 (('taxi', 'driver', 'king'), 2),
 (('driver', 'king', 'comedy'), 2),
 (('masterpiece', 'joaquin', 'phoenix'), 2),
 (('movie', 'the', 'acting'), 2),
 (('dark', 'knight', 'trilogy'), 2),
 (('taxi', 'driver', 'the'), 2),
 (('driver', 'the', 'king'), 2),
 (('the', 'king', 'comedy'), 2),
 (('once', 'upon', 'time'), 2),
 (('upon', 'time', 'holl

1.2 Probablity of Bigrams

In [18]:
freq_dis = nltk.FreqDist(nltk.bigrams(looping))
freq_dict = dict(freq_dis)
for bi_gram in freq_dis:
  print(str(bi_gram) + '-' + str(freq_dict[bi_gram] / looping.count(bi_gram[1])))

Streaming output truncated to the last 5000 lines.
('character', 'complimented')-1.0
('complimented', 'superb')-0.5
('superb', 'soundtrack')-0.25
('soundtrack', 'acting')-0.02564102564102564
('acting', 'a')-0.05555555555555555
('a', 'hauntingly')-1.0
('hauntingly', 'beautiful')-0.5
('beautiful', 'film')-0.013157894736842105
('a', 'definate')-1.0
('definate', 'edit')-1.0
('edit', 'bring')-0.25
('bring', 'part')-0.08333333333333333
('part', 'plssssss')-1.0
('plssssss', 'we')-0.09090909090909091
('we', 'want')-0.07692307692307693
('want', 'it')-0.015151515151515152
('it', 'really')-0.029411764705882353
('really', 'slow')-0.09090909090909091
('slow', 'paced')-1.0
('paced', 'boring')-0.05555555555555555
('boring', 'it')-0.030303030303030304
('it', 'overhyped')-0.2
('overhyped', 'opinion')-0.14285714285714285
('opinion', 'phoenix')-0.016666666666666666
('phoenix', 'convinces')-1.0
('convinces', 'always')-0.25
('always', 'meant')-0.3333333333333333
('meant', 'joker')-0.006535947712418301
('mo

In [24]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [25]:

whole_dataset = ''
index = []
i = 1
for line in df['Lemmatization']:
  whole_dataset = whole_dataset + line
  value = 'Review-' + str(i)
  index.append(value)
  i += 1

from textblob import TextBlob
noun_phrases = []
frequency = []
for line in df['Lemmatization']:
  blob = TextBlob(line)
  for nouns in blob.noun_phrases:
    noun_phrases.append(nouns)
for word in noun_phrases:
  noun_phrases_freq = []
  for line in df['Lemmatization']:
    noun_phrases_freq.append(line.count(word) / whole_dataset.count(word))
  frequency.append(noun_phrases_freq)
noun_phrases_df = pd.DataFrame(frequency).T
noun_phrases_df.columns = list(noun_phrases)
noun_phrases_df.index = index
noun_phrases_df

,e v e r y m o v i e c o m e s t r u l y m,k e s i m p,c t j o,q u i n p e r f o r m,n c e s c e n o g r,p h y b r i l l i,n c e g r o t e s q u e h,u n t i n g c r i n g y h,r d w,t c h t i m e s m e s m e r i z i n g b l i n k e y e w,t c h i n g t r,g i c s e r i o u s l y f u n n y m o m e n t s e m o t i o n,l r o l l e r c o,s t e r s o m e t i m e s m u l t i p l e e m o t i o n s p o p p i n g t i m e f,r t y p i c,c t i o n r i d d l e d p r e d i c t,b l e s u p e r h e r o m o v i e p r o p e r p s y c h o l o g i c,l t h r i l l e r d r,s i n g l e b e s t c h,c t e r d e v e l o p m e n t i e v e r s e e n,t h i s m o v i e f e l t,l o n e i s o l,t e d t r u l y r e l,t e y o u u n d e r s t,n d m o t i v e f e e l s o r r y c h,c t e r,l o t p e o p l e s e e m o v i e t h i n k e n c o u r,g e s v i o l e n c e b u t t r u l y m o v i e e n c o u r,g e e v e r y o n e u s b e c o m e b e t t e r p e r s o n t r e,t e v e r y o n e r e s p e c t m,k e f e e l l i k e b e l o n g w o r l d i n s t e,d m,k i n g f e e l i s o l,t e d,t r u l y m,s t e r p i e c e t h e b e s t h o l l y w o o d f i l m o n e b e s t f i l m s d e c,d e,n d t r u l y b e s t f i l m b r i n g c o m i c b o o k c h i l l i n g l y r e,l i s t i c,l l y r e,...,k e,d v,n t,g e i n c r e d i b l e j o,q u i n p h o e n i x b l o w n g r,t u i t o u s v i o l e n c e m i s p l,c e d v i o l e n c e w r o n g,p p r o,c h j o k e r d i s,p p o i n t i n g c o n n e c t i o n v i e w e r f i l m w,n d e r i n g l o n g p e r i o d s t o n s f o o t,g e h i t c u t t i n g r o o m f l o o r i r e,l l y w,n t e d s t i r r i n g l y c r e e p y c o n n e c t i o n j o,q u i n j o k e r i t i f l o o k i n g j o,q u i n m o m e n t l i k e g l,d i,t o r d i s,p p o i n t e d,w o w m o v i e i,d m i t w h e n i f i r s t h e,r d j o,q u i n p h o e n i x g o i n g j o k e r i s u r p r i s e d i s u r e g o i n g p u l l i h o p i n g g o i n g,n o t h e r c h e e s y r e n d i t i o n f u l l f l u f f u n r e,l i s t i c e x p l,t i o n s j o k e r c,m e o n e,d u l t s o n k e e p n u d g i n g w,k e i p l e,n t l y s u r p r i s e d w e l l t o d d p h i l l i p s v e r s i o n p l,y e d j o,q u i n,c t i n g s u p e r b y o u c o u l d f e e l,n g u i s h c h,c t e r g o i n g r i g h t p o i n t l o s t m i n d j u s t e x c e l l e n t w e l l,c t e d m o v i e,l m o s t f o r g e t d e r i v e d d c c o m i c,t h i s m o v i e p o o r l y d o n e t e l l s s t o r y i t d,r k m o v i e s h o w i n g p r o b l e m m e n t,l i l l n e s s
Review-1,1.0,1.0,1.0,0.333333,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.333333,0.0,0.003472,0.0,0.000000,0.00,...,0.005952,0.000000,0.008108,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.005128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Review-2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.016129,1.0,1.0,1.0,1.0,1.0,0.026667,1.0,0.028169,0.333333,0.0,0.003472,0.0,0.000000,0.00,...,0.008929,0.000000,0.002703,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.005128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Review-3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.040000,0.0,0.042254,0.333333,1.0,0.017361,1.0,0.111111,0.25,...,0.011905,0.000000,0.008108,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.020513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Review-4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.017361,0.0,0.000000,0.00,...,0

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [37]:

import math
sentences = final_df["Lower casing"].values.tolist()
words = set([j for i in sentences for j in i.split(" ")])
tf_idf_df = pd.DataFrame(words, columns=["words"])
count=1

def tf_idf(x,sentence):
  value = sentence.count(x)
  size = len(sentence.split(" "))
  if(value != 0):
    return (value/size)*(math.log(size/value, 10))
  else:
    return 0
    
for sentence in sentences:
  tf_idf_df["Review"+str(count)] = tf_idf_df["words"].apply(lambda x: tf_idf(x,sentence))
  count=count+1
tf_idf_df

,words,Review1,Review2,Review3,Review4,Review5,Review6,Review7,Review8,Review9,Review10,Review11,Review12,Review13,Review14,Review15,Review16,Review17,Review18,Review19,Review20,Review21,Review22,Review23,Review24,Review25,Review26,Review27,Review28,Review29,Review30,Review31,Review32,Review33,Review34,Review35,Review36,Review37,Review38,Review39,...,Review86,Review87,Review88,Review89,Review90,Review91,Review92,Review93,Review94,Review95,Review96,Review97,Review98,Review99,Review100,Review101,Review102,Review103,Review104,Review105,Review106,Review107,Review108,Review109,Review110,Review111,Review112,Review113,Review114,Review115,Review116,Review117,Review118,Review119,Review120,Review121,Review122,Review123,Review124,Review125
0,,-6.905327,-5.182642,-6.244978,-6.538722,-5.069084,-5.334166,-5.738595,-5.628390,-4.744826,0.0,0.0,-5.775853,-6.156653,-5.534701,-5.615037,-4.825652,-6.043934,-4.649361,0.0,-5.208012,-4.537074,-5.000844,-6.099085,-5.492725,0.0,-4.883985,-4.449671,0.0,0.0,-5.114489,-6.287843,-6.288601,-7.352335,-4.821025,-5.114489,-4.668908,-5.331956,0.0,-4.8975,...,-4.842847,-4.622329,-4.721691,-6.129773,-5.310301,-8.128051,-3.988293,-5.325547,-5.444841,-5.492725,-5.112548,-5.222880,-5.858886,-4.511383,-5.586103,-4.931709,-5.316781,-5.244166,-5.09349,-5.425174,-5.240326,-4.903395,0.0,0.0,-5.159552,0.0,-8.128051,-4.520991,-4.908018,-5.749340,-6.283726,0.0,-5.40899,0.0,-4.132858,-4.967975,0.0,-6.828789,-4.942142,-4.856287
1,allows,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,terrible,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.019395,0.0,0.0,0.078406,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,turn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028222,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.027569,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040797,0.0,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,badass,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,

In [39]:
#2.2
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
X_Value = "text directly (rather than e.g. titles and abstracts), and suggests appropriate approaches to doing this, with a focus on the role of natural language processing. The paper also comments on possible connections with data and knowledge retrieval, and concludes by emphasizing the importance of rigorous"
X_after_tokens = word_tokenize(X_Value)
stop = stopwords.words('english') 
X_after_stop_words = {word for word in X_after_tokens if not word in stop}

def cosine_value(X_after_stop_words, Y_after_stop_words):
  cosine = 0
  list1 = []
  list2 = []
  rvector = X_after_stop_words.union(Y_after_stop_words)  
  for w in rvector: 
    if w in X_after_stop_words: list1.append(1)
    else: list1.append(0) 
    if w in X_after_stop_words: list2.append(1) 
    else: list2.append(0) 
  for i in range(len(rvector)): 
    cosine += list1[i]*list2[i] 
  cv = cosine / float((sum(list1)*sum(list2))**0.5)
  return cv

cosine_similarity = []
for i in df['Lower casing']:
  Y_after_tokens = word_tokenize(i)
  Y_after_stop_words = {word for word in Y_after_tokens if not word in stop}
  cosine_similarity.append(cosine_value(X_after_stop_words, Y_after_stop_words))
cs_df = pd.DataFrame(list(zip(df['Lower casing'], cosine_similarity)), columns = ['Abstracts', 'Cosine Similarity'])
cs_df

,Abstracts,Cosine Similarity
0,every movie comes truly makes impact joaquin p...,1.0
1,this movie felt alone isolated truly relate yo...,1.0
2,truly masterpiece the best hollywood film one ...,1.0
3,joaquin phoenix gives tour de force performanc...,1.0
4,most time movies anticipated like end falling ...,1.0
...,...,...
120,i didnt really know much movie i hadnt seen tr...,1.0
121,,1.0
122,film directed produced poorly opportunity take...,1.0
123,wow movie i admit when i first heard joaquin p...,1.0


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# The GitHub link of your final csv file

# Link: 
https://github.com/SaiMudigonda/Saikumar_INFO5731_Spring2021/blob/main/Reviews%20for%20Sentiment%20Analysis.csv